<a href="https://colab.research.google.com/github/raghulprashath/Time-Series-Analysis/blob/master/Market_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Required Headers
import math
from sklearn.preprocessing import MinMaxScaler                      # For Scaling the Dataset
from tensorflow.keras import Sequential                             # to Sequentialise the dataset for modelling
from tensorflow.keras.layers import LSTM,Dense,Dropout,Activation   # LSTM and Dense for Time-series Analysis
import plotly.graph_objects as go                                   # plotly for plotting the timeseries
import numpy as np                                
import pandas as pd
import requests
import json
import openpyxl 
from datetime import datetime,date,timedelta                        # For manual date operations

In [0]:
# Syncing the Excel File
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
downloaded = drive.CreateFile({'id':"1ukO0Ls34dnqwsnJ_xZAfC4ASBLCEtfbN"}) 
downloaded.GetContentFile('Reference.xlsx')
downloaded = drive.CreateFile({'id':"1W28WyCwUF-0d0lYoedtVgp6fw4-4okpX"})
downloaded.GetContentFile('Finance Analysis.xlsx')
downloaded = drive.CreateFile({'id':"1qh_2-pVXy79Bv-Wze4d8fq3GaJUDk5zl"}) 
downloaded.GetContentFile('stockvbs.vbs')
downloaded = drive.CreateFile({'id':"1tLo8-VVKFc-gYOX6aT40fZM9UfmLd2Hq"}) 
downloaded.GetContentFile('forexvb.vbs')
downloaded = drive.CreateFile({'id':"1C3es_3aZ4gRbj5FuvbLauvqk_6QBR4sc"}) 
downloaded.GetContentFile('ccvbs.vbs')

In [0]:
# Stock function - Gets input parameter as the company name and gets the real time data
# Alpha vantage Api and using json, split the file into corresponding dictionary for processing.
# close list - Contains the list of close values   
# dates list - Contains date of all values
# close_list list - Contains list of list of each close value
# prediction_list list - Contains the result of test data predictions
# x_train list - This contains the X_train scaled data for preprocessing and modeeling
# y_train list - This is useful in calculating Rmse

def Stock(company,densevalue=25):
  headers=list()
  close=list()
  dates=list()
  close_list=list()
  c_list=list()
  prediction_list=list()
  x_train=list()
  y_train=list()
  train=list()
  valid=list()

  inputticker = company 
  url = 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol='+inputticker+'&datatype=json&outputsize=full&apikey=UTA722LI2Q27OJRL'
  df = dict(requests.get(url).json())
  pastd = 60
  for keys in df['Time Series (Daily)']:
      if(int(datetime.strptime(keys,'%Y-%m-%d').date().strftime("%Y")) >= int(date.today().strftime("%Y"))-5):
        dates.append(datetime.strptime(keys,'%Y-%m-%d').date())
        if keys not in headers:
            headers.append(df['Time Series (Daily)'][keys])
        close.append(float(df['Time Series (Daily)'][keys]['4. close']))

  dates.reverse()
  close.reverse()
  for no,data in enumerate(headers):
      c_list.append(float(data['4. close']))
      close_list.append(c_list)
      c_list=[]

  close_list.reverse()
  dataset=np.asarray(close_list)
  training_data_len = int(len(dataset)*.80)
  predict(inputticker+' Stocks',close_list,dataset,training_data_len,x_train,y_train,dates,train,valid,prediction_list,close,pastd,densevalue)
  file(dates,training_data_len,dataset,close,inputticker+' Stocks',prediction_list,'Stock')

In [0]:
def Cc(ic,densevalue=25):
  headers=list()
  close=list()
  dates=list()
  close_list=list()
  c_list=list()
  prediction_list=list()
  x_train=list()
  y_train=list()  
  train=list()
  valid=list()

  inputcurrency = ic 
  url = 'https://www.alphavantage.co/query?function=DIGITAL_CURRENCY_DAILY&symbol=BTC&market='+inputcurrency+'&outputsize=full&apikey=UTA722LI2Q27OJRL'
  df = dict(requests.get(url).json())
  pastd = 60
  for keys in df['Time Series (Digital Currency Daily)']:
      if(int(datetime.strptime(keys,'%Y-%m-%d').date().strftime("%Y")) >= int(date.today().strftime("%Y"))-5):
        dates.append(datetime.strptime(keys,'%Y-%m-%d').date())
        if keys not in headers:
            headers.append(df['Time Series (Digital Currency Daily)'][keys])
        close.append(float(df['Time Series (Digital Currency Daily)'][keys]['4a. close ('+inputcurrency+')']))

  dates.reverse()
  close.reverse()
  for no,data in enumerate(headers):
      c_list.append(float(data['4a. close ('+inputcurrency+')']))
      close_list.append(c_list)
      c_list=[]

  close_list.reverse()
  dataset=np.asarray(close_list)
  training_data_len = int(len(dataset)*.80)
  predict(inputcurrency+' CrytoCurrency',close_list,dataset,training_data_len,x_train,y_train,dates,train,valid,prediction_list,close,pastd,densevalue)
  file(dates,training_data_len,dataset,close,inputcurrency+' CrytoCurrency',prediction_list,'CrytoCurrency')

In [0]:
def Forex(cur1,cur2,densevalue=25):
  headers=list()
  close=list()
  dates=list()
  close_list=list()
  c_list=list()
  prediction_list=list()
  x_train=list()
  y_train=list()  
  train=list()
  valid=list()

  url = 'https://www.alphavantage.co/query?function=FX_DAILY&from_symbol='+cur1+'&to_symbol='+cur2+'&outputsize=full&apikey=UTA722LI2Q27OJRL'
  df = dict(requests.get(url).json())
  pastd = 60
  for keys in df['Time Series FX (Daily)']:
      if(int(datetime.strptime(keys,'%Y-%m-%d').date().strftime("%Y")) >= int(date.today().strftime("%Y"))-5):
        dates.append(datetime.strptime(keys,'%Y-%m-%d').date())
        if keys not in headers:
            headers.append(df['Time Series FX (Daily)'][keys])
        close.append(float(df['Time Series FX (Daily)'][keys]['4. close']))

  dates.reverse()
  close.reverse()
  for no,data in enumerate(headers):
      c_list.append(float(data['4. close']))
      close_list.append(c_list)
      c_list=[]

  close_list.reverse()
  dataset=np.asarray(close_list)
  training_data_len = int(len(dataset)*.95)
  predict('Forex '+cur1+' to '+cur2,close_list,dataset,training_data_len,x_train,y_train,dates,train,valid,prediction_list,close,pastd,densevalue)
  file(dates,training_data_len,dataset,close,cur1+' to '+cur2,prediction_list,'Forex')

In [0]:
# Predict Function Scales the data in (0,1) using MinMaxScaller
# Then we Predict the data using LSTM and Dense.
# LSTM is Least Short term method (Deep learning). The input for LSTM is the scaled trained data
# This scaled data is segrated into parts of a value by which we actually predict.
# In this case I use pastd=60, So I am going to predict it with past 60 days data.
# Then finally fitting the data to get the right model 

def predict(title,close_list,dataset,training_data_len,x_train,y_train,dates,train,valid,prediction_list,close,pastd,densevalue):
  
  scaler = MinMaxScaler(feature_range=(0,1))
  scaled_data = scaler.fit_transform(dataset)
  train_data = scaled_data[0:training_data_len,:]
  for i in range(pastd,len(train_data)):
      x_train.append(train_data[i-pastd:i,0])
      y_train.append(train_data[i, 0])

  x_train= np.array(x_train)
  y_train= np.array(y_train)
  x_train = np.reshape(x_train,(x_train.shape[0],x_train.shape[1], 1))

  #Model the LSTM 

  model = Sequential()
  model.add(LSTM(units=15, return_sequences=True,input_shape=(x_train.shape[1], 1)))
  model.add(LSTM(units=15, return_sequences=False))
  model.add(Dense(densevalue))
  model.add(Dense(1))
  model.compile(optimizer='rmsprop', loss='mse')
  model.fit(x_train,y_train,batch_size=1, epochs=1)

  test_data = scaled_data[training_data_len-pastd:]
  x_test = []
  y_test = dataset[training_data_len:, :]
  for i in range(pastd, len(test_data)):
      x_test.append(test_data[i-pastd:i,0])
  x_test = np.array(x_test)
  x_test = np.reshape(x_test,(x_test.shape[0],x_test.shape[1], 1))
  predictions = model.predict(x_test)
  predictions = scaler.inverse_transform(predictions)
  rmse = np.sqrt(np.mean( predictions -y_test)**2 )
  train = close[:training_data_len]
  valid = close[training_data_len:]
  for i in range(len(predictions)):
      prediction_list.append(predictions[i][0])

  #Down here is the code for finding the next 30 days prediction of data

  count=60
  lastdate=dates[-1]
  while(count!=0):
    last = np.asarray(close_list[-pastd:])
    lasts = scaler.transform(last)
    x_test = []
    x_test.append(lasts)
    x_test = np.array(x_test)
    x_test = np.reshape(x_test,(x_test.shape[0],x_test.shape[1], 1))
    prediction = model.predict(x_test)
    prediction = scaler.inverse_transform(prediction)
    close_list.append(prediction[0])
    close.append(prediction[0][0])
    dates.append(lastdate+timedelta(60-count+1))
    count-=1
  graphplot(dates,train,valid,prediction_list,close,pastd,training_data_len,title)

In [0]:
# Graphplot function plots the graph for the given data.
def graphplot(dates,train,valid,prediction_list,close,pastd,training_data_len,title):
  fig = go.Figure()
  fig.add_trace(go.Scatter(x=dates[:training_data_len], y=train,mode='lines',name='Train Data'))
  fig.add_trace(go.Scatter(x=dates[training_data_len:], y=valid,mode='lines',name='Test Data'))
  fig.add_trace(go.Scatter(x=dates[training_data_len:], y=prediction_list,mode='lines+markers',name='Predicted Data'))
  fig.add_trace(go.Scatter(x=dates[-60:], y=close[-60:],mode='lines+markers',name='Future Data'))
  fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
                      dict(count=1,
                           label='1y',
                           step='year',
                           stepmode='todate'),
                      dict(count=2,
                           label='2y',
                           step='year',
                           stepmode='todate'),
                      dict(count=3,
                           label='3y',
                           step='year',
                           stepmode='todate'),
                      dict(step='all')
        ])
    )
  )
  fig.update_layout(
    title={'text': title,
           'x':0.5,
           'xanchor': 'center',
           'yanchor': 'top'},
    xaxis_title="Date",
    yaxis_title="Close Values",
  )
  fig.show()

In [0]:
# These file function stores these records in Excel and we are able to view the record
# as a beautiful dashboard. 
def file(dates,training_data_len,dataset,close,title,prediction_list,filename):
  xfile = openpyxl.load_workbook("Reference.xlsx")
  sheet1 = xfile[filename]

  for row in sheet1['A1:E3000']:
    for cell in row:
      cell.value = None

  sheet1['A1']='Date'
  sheet1['B1']='Close'
  sheet1['C1']='Test'
  sheet1['D1']='Predicted'
  sheet1['E1']='True'
  sheet1['F1']= title
  for no in range(len(dates)):    
      sheet1['A'+str(no+2)]=dates[no]
      if(no == len(dataset)):
        if((round(float(close[no]),4)>round(float(close[no-1]),4))):
          sheet1['F2']='High'
        else:
          sheet1['F2']='Low'
      if(no<len(dataset)):
          sheet1['B'+str(no+2)]=float(close[no])
      else:
          sheet1['D'+str(no+2)]=float(close[no])
      if(no>=training_data_len and no<len(dataset)):
          sheet1['C'+str(no+2)] = round(float(prediction_list[no-training_data_len-1]),4)
          if(round(float(close[no]),4) == round(float(prediction_list[no-training_data_len-1]),4)):
            sheet1['E'+str(no+2)] = 'True'
          elif((round(float(close[no-1]),4)-round(float(prediction_list[no-training_data_len-1]),4))>0 and (round(float(close[no-1]),4)-round(float(close[no]),4)>0)):
            sheet1['E'+str(no+2)] = 'True'
          elif((round(float(close[no-1]),4)-round(float(prediction_list[no-training_data_len-1]),4))<0 and (round(float(close[no-1]),4)-round(float(close[no]),4)<0)):
            sheet1['E'+str(no+2)] = 'True'
          else:
            sheet1['E'+str(no+2)] = 'False'
  xfile.save("Reference.xlsx")

In [0]:
#@title Forex { display-mode: "form",run: "auto" }
Currency1= "AUD" #@param ["AED","AFN","ALL","AMD","ANG","AOA","ARS","AUD","AWG","AZN","BAM","BBD","BDT","BGN","BHD","BIF","BMD","BND","BOB","BRL","BSD","BTN","BWP","BZD","CAD","CDF","CHF","CLF","CLP","CNH","CNY","COP","CUP","CVE","CZK","DJF","DKK","DOP","DZD","EGP","ERN","ETB","EUR","FJD","FKP","GBP","GEL","GHS","GIP","GMD","GNF","GTQ","GYD","HKD","HNL","HRK","HTG","HUF","IDR","ILS","INR","IQD","IRR","ISK","JEP","JMD","JOD","JPY","KES","KGS","KHR","KMF","KPW","KRW","KWD","KYD","KZT","LAK","LBP","LKR","LRD","LSL","LYD","MAD","MDL","MGA","MKD","MMK","MNT","MOP","MRO","MRU","MUR","MVR","MWK","MXN","MYR","MZN","NAD","NGN","NOK","NPR","NZD","OMR","PAB","PEN","PGK","PHP","PKR","PLN","PYG","QAR","RON","RSD","RUB","RUR","RWF","SAR","SBD","SCR","SDG","SEK","SGD","SHP","SLL","SOS","SRD","SYP","SZL","THB","TJS","TMT","TND","TOP","TRY","TTD","TWD","TZS","UAH","UGX","USD","UYU","UZS","VND","VUV","WST","XAF","XAG","XAU","XCD","XDR","XOF","XPF","YER","ZAR","ZMW","ZWL"]
Currency2= "CNY" #@param ["AED","AFN","ALL","AMD","ANG","AOA","ARS","AUD","AWG","AZN","BAM","BBD","BDT","BGN","BHD","BIF","BMD","BND","BOB","BRL","BSD","BTN","BWP","BZD","CAD","CDF","CHF","CLF","CLP","CNH","CNY","COP","CUP","CVE","CZK","DJF","DKK","DOP","DZD","EGP","ERN","ETB","EUR","FJD","FKP","GBP","GEL","GHS","GIP","GMD","GNF","GTQ","GYD","HKD","HNL","HRK","HTG","HUF","IDR","ILS","INR","IQD","IRR","ISK","JEP","JMD","JOD","JPY","KES","KGS","KHR","KMF","KPW","KRW","KWD","KYD","KZT","LAK","LBP","LKR","LRD","LSL","LYD","MAD","MDL","MGA","MKD","MMK","MNT","MOP","MRO","MRU","MUR","MVR","MWK","MXN","MYR","MZN","NAD","NGN","NOK","NPR","NZD","OMR","PAB","PEN","PGK","PHP","PKR","PLN","PYG","QAR","RON","RSD","RUB","RUR","RWF","SAR","SBD","SCR","SDG","SEK","SGD","SHP","SLL","SOS","SRD","SYP","SZL","THB","TJS","TMT","TND","TOP","TRY","TTD","TWD","TZS","UAH","UGX","USD","UYU","UZS","VND","VUV","WST","XAF","XAG","XAU","XCD","XDR","XOF","XPF","YER","ZAR","ZMW","ZWL"]
Forex(Currency1,Currency2,20)  

In [0]:
#@title CrptoCurrency { display-mode: "form",run: "auto" }
Currency= "CNY" #@param ["AED","AFN","ALL","AMD","ANG","AOA","ARS","AUD","AWG","AZN","BAM","BBD","BDT","BGN","BHD","BIF","BMD","BND","BOB","BRL","BSD","BTN","BWP","BZD","CAD","CDF","CHF","CLF","CLP","CNH","CNY","COP","CUP","CVE","CZK","DJF","DKK","DOP","DZD","EGP","ERN","ETB","EUR","FJD","FKP","GBP","GEL","GHS","GIP","GMD","GNF","GTQ","GYD","HKD","HNL","HRK","HTG","HUF","IDR","ILS","INR","IQD","IRR","ISK","JEP","JMD","JOD","JPY","KES","KGS","KHR","KMF","KPW","KRW","KWD","KYD","KZT","LAK","LBP","LKR","LRD","LSL","LYD","MAD","MDL","MGA","MKD","MMK","MNT","MOP","MRO","MRU","MUR","MVR","MWK","MXN","MYR","MZN","NAD","NGN","NOK","NPR","NZD","OMR","PAB","PEN","PGK","PHP","PKR","PLN","PYG","QAR","RON","RSD","RUB","RUR","RWF","SAR","SBD","SCR","SDG","SEK","SGD","SHP","SLL","SOS","SRD","SYP","SZL","THB","TJS","TMT","TND","TOP","TRY","TTD","TWD","TZS","UAH","UGX","USD","UYU","UZS","VND","VUV","WST","XAF","XAG","XAU","XCD","XDR","XOF","XPF","YER","ZAR","ZMW","ZWL"]
Cc(Currency,20)

In [0]:
#@title Stock { display-mode: "form",run: "auto" }
Company = "BEST" #@param {type:"string"}
Stock(Company,12)